In [453]:
from IPython.display import clear_output
import time

In [454]:
def load_file_lines(filename):
    lines = []
    file = open(filename)
    for line in file:
        lines.append(line.rstrip("\n"))
    file.close()
    return lines

datas = load_file_lines("./pawel.txt")
# datas = load_file_lines("./test.txt")
# datas

In [455]:
class Item:
    def __init__(self, worry_level_str):
        self.worry_level = worry_level_str
    
    def __repr__(self):
        return f"{self.worry_level} "
    
    def multiply_worry(self, operation, number):
        if operation == "+":
            self.worry_level = self.worry_level + number
        elif operation == "**":
            self.worry_level = self.worry_level ** number
        else:
            self.worry_level = self.worry_level * number
        
    def get_bored(self, divide_by):
        self.worry_level = self.worry_level // divide_by

In [456]:
class Monkey:
    def __init__(self, lines, bored_divide_by):
        self.name = lines[0]
        self.bored_divide_by = bored_divide_by
        self.items = self.line_into_items(lines[1])
        self.number_of_inspected_items = 0
        self.operation, self.change_number = self.line_into_operation(lines[2])
        self.test_divisible_by = self.line_into_test(lines[3])
        self.pass_to_monkey_if_true = self.line_into_monkey_to_pass_to(lines[4])
        self.pass_to_monkey_if_false = self.line_into_monkey_to_pass_to(lines[5])
        
    def who_to_throw_to(self, item):
#         print(f"{item}")
        item.multiply_worry(self.operation, self.change_number)
#         print(f"{item}")
        item.get_bored(self.bored_divide_by)
#         print(f"{item}")

        if item.worry_level % self.test_divisible_by == 0:
            return self.pass_to_monkey_if_true
        else:
            return self.pass_to_monkey_if_false
    
    def inspect_next_item(self):
        self.number_of_inspected_items += 1
        if len(self.items) == 0:
            return [None, None]
        else:
            throw_to = self.who_to_throw_to(self.items[0])
            item = self.items.pop(0);
            return [item, throw_to]
    
    def accept_item(self, new_item):
        self.items.append(new_item)
    
    def line_into_items(self, line):
        #   Starting items: 79, 98
        return [
            Item(int(number))
           for number in line.replace("  Starting items: ","").split(", ")]

    def line_into_operation(self, line):
        #   Operation: new = old * 19
        results = line.replace("  Operation: new = old ","").split(" ")
        if results[1] == "old":
            return ["**", 2] # new = old * old ... basically means square it
        else:
            return [results[0], int(results[1])]

            
    def line_into_test(self, line):
        #   Test: divisible by 23
         return int(line.replace("  Test: divisible by ",""))
        
    def line_into_monkey_to_pass_to(self, line):
        #   Test: divisible by 23
         return int(line.replace("true","").replace("false","").replace("    If : throw to monkey ",""))
        
    def __repr__(self):
#         description = f"{self.name} has {self.items}. Op {self.operation} {self.change_number} "
#         description += f"if divs by {self.test_divisible_by} give to {self.pass_to_monkey_if_true}"
#         description += f" else to {self.pass_to_monkey_if_false}"
#         return description
        return f"{self.name} has {len(self.items)} \t and inspected {self.number_of_inspected_items}"
    
        
# Monkey 0:
#   Starting items: 79, 98
#   Operation: new = old * 19
#   Test: divisible by 23
#     If true: throw to monkey 2
#     If false: throw to monkey 3

In [457]:
def turn_lines_into_monkeys(lines, bored_divide_by):
    monkeys = []
    this_monkey_lines = []
    for line in lines:
        if line == '':
            pass
        else:
            this_monkey_lines.append(line)
            
        if (line[0:30] == "    If false: throw to monkey "):
            monkeys.append(Monkey(this_monkey_lines, bored_divide_by))
            this_monkey_lines = []
    return monkeys

monkeys = turn_lines_into_monkeys(datas,3)

In [458]:
def activate_monkeys(monkeys):
    for monkey in monkeys:
        for item_number in range(len(monkey.items)):
            item, new_monkey = monkey.inspect_next_item()
            if(new_monkey != None):
#                 print(f"{monkey.name} will pass to {new_monkey}")
                monkeys[new_monkey].accept_item(item)

In [459]:
# print(monkeys[0].items[0] )
item, new_monkey = monkeys[0].inspect_next_item()
print(item, new_monkey)

151  4


In [460]:
monkeys = turn_lines_into_monkeys(datas,3)

for monkey in monkeys:
    print(monkey)

# one turn
for index in range(20):
    activate_monkeys(monkeys)

for monkey in monkeys:
    print(monkey)

Monkey 0: has 5 	 and inspected 0
Monkey 1: has 7 	 and inspected 0
Monkey 2: has 3 	 and inspected 0
Monkey 3: has 4 	 and inspected 0
Monkey 4: has 1 	 and inspected 0
Monkey 5: has 2 	 and inspected 0
Monkey 6: has 6 	 and inspected 0
Monkey 7: has 8 	 and inspected 0
Monkey 0: has 0 	 and inspected 51
Monkey 1: has 12 	 and inspected 247
Monkey 2: has 7 	 and inspected 204
Monkey 3: has 17 	 and inspected 201
Monkey 4: has 0 	 and inspected 249
Monkey 5: has 0 	 and inspected 216
Monkey 6: has 0 	 and inspected 53
Monkey 7: has 0 	 and inspected 47


In [461]:
def item_ever_counts_for_monkeys(monkeys):
    return [ monkey.number_of_inspected_items
        for monkey in monkeys]

item_evers = item_ever_counts_for_monkeys(monkeys)
print(item_evers)
largest_two = sorted(item_evers)[-2:]

largest_two[0] * largest_two[1] 
# part 1 was 61503. test was 10605

[51, 247, 204, 201, 249, 216, 53, 47]


61503

# part 2

In [513]:
class ItemWithMultipliers:
    
    def __init__(self, worry_level, multipliers):
        self.levels = {}
        for multiplier in multipliers:
            self.levels[multiplier] = worry_level % multiplier
    
    def __repr__(self):
        return f"{self.levels} "
    
    def multiply_worry(self, operation, number):
        for divider, leftover in self.levels.items():
            if operation == "+":
                increased_number = (leftover + number) 
            elif operation == "**":
                increased_number =(leftover ** number) 
            else:
                increased_number =(leftover * number) 
            self.levels[divider] = increased_number % divider
    def get_bored(self, divide_by):
        self.multiply_worry('*',divide_by)
# item = ItemMulti(10,[2,3,4,5])
# print(item)
# item.multiply_worry("+",1)
# print(item)

In [514]:
class Monkey:
    def __init__(self, lines, bored_divide_by):
        self.name = lines[0]
        self.bored_divide_by = bored_divide_by
        self.items_raw = self.line_into_items(lines[1])
        self.items = []
        self.number_of_inspected_items = 0
        self.operation, self.change_number = self.line_into_operation(lines[2])
        self.test_divisible_by = self.line_into_test(lines[3])
        self.pass_to_monkey_if_true = self.line_into_monkey_to_pass_to(lines[4])
        self.pass_to_monkey_if_false = self.line_into_monkey_to_pass_to(lines[5])
        
    def who_to_throw_to(self, item):
        item.multiply_worry(self.operation, self.change_number)
        item.get_bored(self.bored_divide_by)

        if item.levels[self.test_divisible_by]  == 0:
            return self.pass_to_monkey_if_true
        else:
            return self.pass_to_monkey_if_false
    
    def inspect_next_item(self):
        self.number_of_inspected_items += 1
        if len(self.items) == 0:
            return [None, None]
        else:
            throw_to = self.who_to_throw_to(self.items[0])
            item = self.items.pop(0);
            return [item, throw_to]
    
    def improve_items_with_dividers(self, dividers):
        self.items = [ItemWithMultipliers(item.worry_level, dividers) 
                      for item in self.items_raw]
    
    def accept_item(self, new_item):
        self.items.append(new_item)
    
    def line_into_items(self, line):
        #   Starting items: 79, 98
        return [
            Item(int(number))
           for number in line.replace("  Starting items: ","").split(", ")]

    def line_into_operation(self, line):
        #   Operation: new = old * 19
        results = line.replace("  Operation: new = old ","").split(" ")
        if results[1] == "old":
            return ["**", 2] # new = old * old ... basically means square it
        else:
            return [results[0], int(results[1])]

            
    def line_into_test(self, line):
        #   Test: divisible by 23
         return int(line.replace("  Test: divisible by ",""))
        
    def line_into_monkey_to_pass_to(self, line):
        #   Test: divisible by 23
         return int(line.replace("true","").replace("false","").replace("    If : throw to monkey ",""))
        
    def __repr__(self):
#         description = f"{self.name} has {self.items}. Op {self.operation} {self.change_number} "
#         description += f"if divs by {self.test_divisible_by} give to {self.pass_to_monkey_if_true}"
#         description += f" else to {self.pass_to_monkey_if_false}"
#         return description
        return f"{self.name} has {len(self.items)} \t and inspected {self.number_of_inspected_items}"
    
        
# Monkey 0:
#   Starting items: 79, 98
#   Operation: new = old * 19
#   Test: divisible by 23
#     If true: throw to monkey 2
#     If false: throw to monkey 3

In [515]:
# monkeys = turn_lines_into_monkeys(datas,3) #part 1
monkeys = turn_lines_into_monkeys(datas,1)


In [516]:
def get_monkey_dividers(monkeys):
    return [ monkey.test_divisible_by
        for monkey in monkeys]



In [517]:
# for monkey in monkeys:
#     monkey.improve_items_with_dividers(get_monkey_dividers(monkeys))
    
# monkeys[0].items

In [518]:
def activate_monkeys(monkeys):
    for monkey in monkeys:
        for item_number in range(len(monkey.items)):
            item, new_monkey = monkey.inspect_next_item()
            if(new_monkey != None):
#                 print(f"{monkey.name} will pass to {new_monkey}")
                monkeys[new_monkey].accept_item(item)

In [519]:
print(monkeys[0].items)
activate_monkeys(monkeys)
print(monkeys[0].items)


[]
[]


In [520]:
import time

monkeys = turn_lines_into_monkeys(datas,1)

for monkey in monkeys:
    monkey.improve_items_with_dividers(get_monkey_dividers(monkeys))
    
for monkey in monkeys:
    print(monkey)

t = time.process_time()
    
# one turn
# number_of_turns = 20 # part 1
number_of_turns = 10000

for index in range(number_of_turns):
    print('.', end="")
    if (index % 50 == 0):
        print(index)
    activate_monkeys(monkeys)

elapsed_time = time.process_time() - t

print(f"elapsed_time {elapsed_time}")
print(f"elapsed_time minutes {elapsed_time // 60}")

for monkey in monkeys:
    print(monkey)
    
# 1000 rounds  elapsed_time 1469.4257580000003

Monkey 0: has 5 	 and inspected 0
Monkey 1: has 7 	 and inspected 0
Monkey 2: has 3 	 and inspected 0
Monkey 3: has 4 	 and inspected 0
Monkey 4: has 1 	 and inspected 0
Monkey 5: has 2 	 and inspected 0
Monkey 6: has 6 	 and inspected 0
Monkey 7: has 8 	 and inspected 0
.0
..................................................50
..................................................100
..................................................150
..................................................200
..................................................250
..................................................300
..................................................350
..................................................400
..................................................450
..................................................500
..................................................550
..................................................600
..................................................650
........................

..................................................7650
..................................................7700
..................................................7750
..................................................7800
..................................................7850
..................................................7900
..................................................7950
..................................................8000
..................................................8050
..................................................8100
..................................................8150
..................................................8200
..................................................8250
..................................................8300
..................................................8350
..................................................8400
..................................................8450
..................................................8500
..........

In [521]:
def item_ever_counts_for_monkeys(monkeys):
    return [ monkey.number_of_inspected_items
        for monkey in monkeys]

item_evers = item_ever_counts_for_monkeys(monkeys)
print(item_evers)
largest_two = sorted(item_evers)[-2:]

largest_two[0] * largest_two[1] 
# part 1 was 61503, part 2 was 14081365540 

[45924, 131087, 87798, 95174, 107420, 96174, 74384, 41580]


14081365540